<a href="https://colab.research.google.com/github/ubern-mia/bme-labs/blob/main/Session3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 3

Today, we will look into what features a classifier deems important for the decision it takes.

First, load the data againg (the same one you used last time).

In [ ]:
from google.colab import files
! cd "/content"
uploaded = files.upload()

measurements = "/content/" + list(uploaded.keys())[0]

As last time, define a training and testing split and define the feature columns. Then we will use a random forest classifier for the prediction. Instead of the cross-validation as last time, we use the full training set.

In [ ]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, balanced_accuracy_score

# define a mapping of the disease encoding back to strings
diseasestatus = {0: "Healthy", 1: "ASD", 2: "Epilepsy"}

measurements = pd.read_csv("/content/fsfaststats.csv")

# map the disease name to the encoding and make sure the age is a float
measurements["Disease"] = [diseasestatus[e] for e in measurements["Disease"]]
measurements["Age"] = [float(e) for e in measurements["Age"]]

# X is the feature matrix we feed to the classifier, i.e. the measurements and the age of the subject
features = list(set(measurements.columns) - set(["Subject", "Disease"]))
X = measurements.loc[:, features]
y = measurements["Disease"]

# Reserve 20% of the data for testing
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in sss.split(X, y):
  print("Training set size: " + str(len(train_index)))
  print("Test set size: " + str(len(test_index)))

X_train = X.iloc[train_index, :]
y_train = y.iloc[train_index]

X_test = X.iloc[test_index, :]
y_test = y.iloc[test_index]

# Plug in the parameter settings that work well according to your experiments
clf = RandomForestClassifier(n_estimators=200, max_depth=None, random_state=42)

# Train the classifier and apply it to the test set
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

With this trained classifier, we can now get information on which features are the most important ones:

In [ ]:
featimportances = clf.feature_importances_

# sort these in descending order and see which features they correspond to
sortidx = (-featimportances).argsort()
importances_sorted = featimportances[sortidx]
featurenames_sorted = X_train.columns[sortidx]
print(importances_sorted)
print(featurenames_sorted)

Let's plot the feature importances. Since they are so many, limit it to the top 15 features.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

n_most_important = 20
sns.barplot(x=importances_sorted[:n_most_important], 
            y=featurenames_sorted[:n_most_important])
plt.title("Random forest feature importance")
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.show()

Now you know roughly which features seem to be the most important ones. Now try to see if you can already see the importance by looking at them individually. For this, you can (as we did during the previous sessions), plot the features. Replace the example below where the left hippocampus is selected with the most important feature. Please do this for the top three according to your previous findings.

In [ ]:
plotcols = list(set(measurements.columns) - set(["Age", "Subject", "Disease"]))
feature = "Left-Hippocampus"

sns.lmplot(x="Age", y=feature, hue="Disease", data=measurements, ci=95)
plt.title(feature)
plt.show()